In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from PIL import Image


In [2]:
directory = 'fire_dataset/fire_images/'
fire = np.array([np.array(Image.open(directory+filename)) for filename in os.listdir(directory)],dtype = object)


In [3]:
directory = 'fire_dataset/non_fire_images/'
non_fire = np.array([np.array(Image.open(directory+filename)) for filename in os.listdir(directory)],dtype = object)

In [4]:
# np.save('fire.npy',fire, allow_pickle=True)
# np.save('non_fire.npy',non_fire, allow_pickle = True)
# fire=np.load('fire.npy',allow_pickle= True)
# non_fire=np.load('non_fire.npy', allow_pickle= True)

In [5]:
df = pd.DataFrame(columns=['path','label'])

for dirname, _, filenames in os.walk('fire_dataset/fire_images'):
    for filename in filenames:
        # print(os.path.join(dirname, filename))
        df = df.append(pd.DataFrame([[os.path.join(dirname, filename),'fire']],columns=['path','label']))

for dirname, _, filenames in os.walk('fire_dataset/non_fire_images'):
    for filename in filenames:
        df = df.append(pd.DataFrame([[os.path.join(dirname, filename),'non_fire']],columns=['path','label']))
        # print(os.path.join(dirname, filename))

df = df.sample(frac=1).reset_index(drop=True)
df.head(10)

,path,label
0,fire_dataset/non_fire_images\non_fire.91.png,non_fire
1,fire_dataset/non_fire_images\non_fire.93.png,non_fire
2,fire_dataset/fire_images\fire.392.png,fire
3,fire_dataset/fire_images\fire.582.png,fire
4,fire_dataset/fire_images\fire.437.png,fire
5,fire_dataset/fire_images\fire.473.png,fire
6,fire_dataset/fire_images\fire.20.png,fire
7,fire_dataset/fire_images\fire.423.png,fire
8,fire_dataset/fire_images\fire.33.png,fire
9,fire_dataset/non_fire_images\non_fire.132.png,non_fire


In [6]:
def shaper(row):
    shape = image.load_img(row['path']).size
    row['height'] = shape[1]
    row['width'] = shape[0]
    return row
df = df.apply(shaper,axis=1)
df.head(5)

,path,label,height,width
0,fire_dataset/non_fire_images\non_fire.91.png,non_fire,348,620
1,fire_dataset/non_fire_images\non_fire.93.png,non_fire,1308,2120
2,fire_dataset/fire_images\fire.392.png,fire,501,890
3,fire_dataset/fire_images\fire.582.png,fire,549,976
4,fire_dataset/fire_images\fire.437.png,fire,1000,1355


In [7]:
generator = ImageDataGenerator(
    rotation_range= 20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range = 2,
    zoom_range=0.2,
    rescale = 1/255,
    validation_split=0.2,
)

In [8]:
train_gen = generator.flow_from_dataframe(df,x_col='path',y_col='label',images_size=(256,256),class_mode='binary',subset='training')
val_gen = generator.flow_from_dataframe(df,x_col='path',y_col='label',images_size=(256,256),class_mode='binary',subset='validation')

Found 800 validated image filenames belonging to 2 classes.
Found 199 validated image filenames belonging to 2 classes.


In [9]:
model = Sequential()
model.add(Conv2D(filters=32,kernel_size = (2,2),activation='relu',input_shape = (256,256,3)))
model.add(MaxPool2D())
model.add(Conv2D(filters=64,kernel_size=(2,2),activation='relu'))
model.add(MaxPool2D())
model.add(Conv2D(filters=128,kernel_size=(2,2),activation='relu'))
model.add(MaxPool2D())
model.add(Conv2D(filters=256,kernel_size=(2,2),activation='relu'))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation = 'relu'))
model.add(Dense(1,activation = 'sigmoid'))

In [10]:
from tensorflow.keras.metrics import Recall,AUC
from tensorflow.keras.utils import plot_model

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


In [11]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stoppping = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)
reduce_lr_on_plateau = ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=5)



In [12]:
model.fit(x=train_gen,batch_size=32,epochs=10,validation_data=val_gen,callbacks=[early_stoppping,reduce_lr_on_plateau])



Epoch 1/100
25/25 [==============================] - 32s 1s/step - loss: 0.3694 - accuracy: 0.8313 - val_loss: 0.1380 - val_accuracy: 0.9698
Epoch 2/100
22/25 [=========================>....] - ETA: 3s - loss: 0.1055 - accuracy: 0.9602

KeyboardInterrupt: 

In [ ]:
eval_list = model.evaluate(val_gen,return_dict=True)
for metric in eval_list.keys():
    print(metric+f": {eval_list[metric]:.2f}")

7/7 [==============================] - 4s 608ms/step - loss: 0.0875 - accuracy: 0.9698 - recall: 0.8636 - auc: 0.9952
loss: 0.09
accuracy: 0.97
recall: 0.86
auc: 1.00
